#### Общая информация

Целью исследования является установить зависимость рыночной стоимости объектов недвижимости, от различных факторов.

Для исследования представлена таблица, в которой приведены данные из объявлений о продаже объектов недвижимости, дополненные объективными данными об объектах из карт.

#### Что сделано

Заполнены пропуски, исправлены ошибки, сделаны выводы о ценах предложения на квартиры в зависимости от разных факторов, для разных групп квартир

In [ ]:
import csv
import pandas as pd
data = pd.read_csv('/datasets/real_estate_data.csv', sep= '\t')
print(data.info())

In [ ]:
data['price_bym'] = data['last_price'] / data['total_area'] / 1000
data['price_bym'].astype('int')
data['price_bym'].describe()

In [ ]:
print(data['building_series_id'].unique())

In [ ]:
head_building_series = ['building_series_id', 'total_area', 'rooms']
data_building_series= pd.DataFrame(data = data, columns = (head_building_series))
print(data_building_series.head(5))

In [ ]:
data_building_series = data_building_series.dropna()
print(data_building_series.head())

In [ ]:
data_building_series['building_series_id'] = data_building_series['building_series_id'].str.lower()
data_building_series.sort_values(by='building_series_id', inplace=True)
print(data_building_series['building_series_id'].unique())

In [ ]:
def bldng_srs_id_dplct (building_series_id):
        if building_series_id == "121 (гатчинская)" or building_series_id == "121 гатчинская" or building_series_id == "121(гатчинская)":
            return "121гатчинская"
        elif building_series_id == 'индивид.' or building_series_id == 'индивидуальный проект': 
            return 'индивидуальный'
        elif building_series_id == 'кирп.монолит':
            return 'кирпичный монолит'
        elif building_series_id == 'старый фонд + капремонт':
            return 'старый фонд с кр'
        else:
            return building_series_id
                           
data_building_series['building_series_id'] = data_building_series['building_series_id'].str.replace("серия", '')
data_building_series_tmp = pd.to_numeric(data_building_series['building_series_id'], errors='coerce') 
data_building_series_tmp = data_building_series_tmp.fillna(-1)
data_building_series_tmp = data_building_series_tmp.astype(int)
data_building_series_tmp.loc[data_building_series_tmp == -1] = data_building_series['building_series_id']
data_building_series['building_series_id'] = data_building_series_tmp
data_building_series['building_series_id'] = data_building_series.apply(lambda x: bldng_srs_id_dplct(x['building_series_id']), axis = 1)
data_building_series['building_series_id'].unique()


In [ ]:
data_pivot_building_series_pt = data_building_series.pivot_table(index=['building_series_id', 'rooms'], values='total_area', aggfunc = ['median', 'count'])

print(data_pivot_building_series_pt.loc[['1 лг - 600.11']])
#print(data_pivot_building_series_pt.loc[['600.11']])
print(data_pivot_building_series_pt.loc[['1-лг-600-i']])

In [ ]:
print(data_pivot_building_series_pt.loc[['1-528 кп - 2']])
print(data_pivot_building_series_pt.loc[['1-528-кп-97']])

In [ ]:
print(data_pivot_building_series_pt.loc[[121]])
print(data_pivot_building_series_pt.loc[['121гатчинская']])

In [ ]:
print(data_pivot_building_series_pt.loc[['1лг-6066м']])
print(data_pivot_building_series_pt.loc[[606]])
data_building_series.replace('1лг-6066м.', 606, inplace = True)

In [ ]:
print(data_pivot_building_series_pt.loc[[504]])
print(data_pivot_building_series_pt.loc[['504д']])    

In [ ]:
data_pivot_building_series1 = (data_building_series
                              .query('rooms == 1')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series1 = (data_building_series
                        .query('rooms == 1')
)
data_pivot_building_series1.columns = ['total_area', 'count']
dt_pvt_bldng1 = data_pivot_building_series1.query('count >= 5')
dt_pvt_bldng1_change = data_pivot_building_series1.query('count < 5')

In [ ]:
data_building_series1.head(10)

In [ ]:
rooms1_stat = []
rooms1_stat = dt_pvt_bldng1.index
print(rooms1_stat)

In [ ]:
data_building_series1_select = data_building_series1.query('building_series_id in @rooms1_stat')
print(data_building_series1_select['building_series_id'].unique())

In [ ]:
data_building_series1_select_pvt = data_building_series1_select.pivot_table(index = 'building_series_id', values = 'total_area', aggfunc = ['median', 'describe'])
data_building_series1_select_pvt.set_axis(['median', '25%', '50%', '75%', 'count', 'max', 'mean', 'min', 'std'], axis = 'columns', inplace = True)
data_building_series1_select_pvt

In [ ]:
data_building_series1_select_pvt['low_b'] =  data_building_series1_select_pvt['25%'] - 1.5 * (data_building_series1_select_pvt['75%'] - data_building_series1_select_pvt['25%'])
data_building_series1_select_pvt['high_b'] =  data_building_series1_select_pvt['75%'] + 1.5 * (data_building_series1_select_pvt['75%'] - data_building_series1_select_pvt['25%'])
data_building_series1_select_pvt

In [ ]:
data_pivot_building_series2 = (data_building_series
                              .query('rooms == 2')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series2 = (data_building_series
                        .query('rooms == 2')
)
data_pivot_building_series2.columns = ['total_area', 'count']
data_pivot_building_series2
dt_pvt_bldng2 = data_pivot_building_series2.query('count >= 5')
dt_pvt_bldng2

In [ ]:
def create_building_id_table (data_building_series_r_select, rooms):
    data_building_series_r_select = data_building_series_r_select.query('building_series_id in @rooms')        
    data_building_series_r_select_pvt = data_building_series_r_select.pivot_table(index = 'building_series_id', values = 'total_area', aggfunc = ['median', 'describe'])
    data_building_series_r_select_pvt.set_axis(['median', '25%', '50%', '75%', 'count', 'max', 'mean', 'min', 'std'], axis = 'columns', inplace = True)
    data_building_series_r_select_pvt['low_b'] =  data_building_series_r_select_pvt['25%'] - 1.5 * (data_building_series_r_select_pvt['75%'] - data_building_series_r_select_pvt['25%'])
    data_building_series_r_select_pvt['high_b'] =  data_building_series_r_select_pvt['75%'] + 1.5 * (data_building_series_r_select_pvt['75%'] - data_building_series_r_select_pvt['25%'])
    return data_building_series_r_select_pvt
    
rooms2_stat = []
rooms2_stat = dt_pvt_bldng2.index
data_building_series2_select_pvt = create_building_id_table(data_building_series2, rooms2_stat)
data_building_series2_select_pvt

In [ ]:
data_pivot_building_series3 = (data_building_series
                              .query('rooms == 3')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series3 = (data_building_series
                        .query('rooms == 3')
)
data_pivot_building_series3.columns = ['total_area', 'count']
data_pivot_building_series3
dt_pvt_bldng3 = data_pivot_building_series3.query('count >= 5')
dt_pvt_bldng3

In [ ]:
rooms3_stat = []
rooms3_stat = dt_pvt_bldng3.index
data_building_series3_select_pvt = create_building_id_table(data_building_series3, rooms3_stat)
data_building_series3_select_pvt

In [ ]:
data_pivot_building_series4 = (data_building_series
                              .query('rooms == 4')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series4 = (data_building_series
                        .query('rooms == 4')
)
data_pivot_building_series4.columns = ['total_area', 'count']
data_pivot_building_series4
dt_pvt_bldng4 = data_pivot_building_series4.query('count >= 5')
dt_pvt_bldng4

In [ ]:
data_pivot_building_series4h = (data_building_series
                              .query('rooms > 4')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series4h = (data_building_series
                        .query('rooms > 4')
)
data_pivot_building_series4h.columns = ['total_area', 'count']
data_pivot_building_series4h
dt_pvt_bldng4h = data_pivot_building_series4h.query('count >= 5')
dt_pvt_bldng4h

In [ ]:
data_pivot_building_series4h

In [ ]:
data_pivot_building_series4 = (data_building_series
                              .query('rooms >= 4')
                              .pivot_table(index='building_series_id', values='total_area', aggfunc = ['median', 'count'])                        
)
data_building_series4 = (data_building_series
                        .query('rooms >= 4')
)
data_pivot_building_series4.columns = ['total_area', 'count']
data_pivot_building_series4
dt_pvt_bldng4 = data_pivot_building_series4.query('count >= 5')
dt_pvt_bldng4

In [ ]:
rooms4_stat = []
rooms4_stat = dt_pvt_bldng4.index
data_building_series4_select_pvt = create_building_id_table(data_building_series4, rooms4_stat)
data_building_series4_select_pvt

In [ ]:
dt_pvt_bldng1_index_set = set(dt_pvt_bldng1.index)
dt_pvt_bldng2_index_set = set(dt_pvt_bldng2.index)
dt_pvt_bldng3_index_set = set(dt_pvt_bldng3.index)
dt_pvt_bldng4_index_set = set(dt_pvt_bldng4.index)
dt_pvt_bldng_index_set = dt_pvt_bldng1_index_set.union(dt_pvt_bldng2_index_set, dt_pvt_bldng3_index_set, dt_pvt_bldng4_index_set)
print(dt_pvt_bldng_index_set)

In [ ]:
data_building_series['building_series_id'] = data_building_series['building_series_id'].where(data_building_series['building_series_id'].isin(dt_pvt_bldng_index_set), 'другие')
#data_building_series.groupby('building_series_id').count()
data_building_series.drop(['total_area', 'rooms'], axis='columns', inplace=True)

In [ ]:
data = data.merge(data_building_series, left_index=True, right_index=True, how='left')
data['building_series_id_y'] = data['building_series_id_y'].fillna('замена')
data.drop(['building_series_id_x'], axis='columns', inplace=True)
data.groupby('building_series_id_y').count()

In [ ]:
data['rooms'].unique()

In [ ]:
def change_building_series_id (building_series_id_y, total_area, rooms):
    if building_series_id_y == 'замена':
        if rooms == 1:
            if 46 >= total_area >= 30:
                return '1564812'
            else:
                return 'другие'
        elif rooms == 2:
            if 73 >= total_area >= 43:
                return '1564812'
            else:
                return 'другие'
        elif rooms == 3:
            if 60 >= total_area >= 103:
                return '1564812'
            elif 53.5 >= total_area > 60:
                return '1564801'
            else:
                return 'другие'
        elif rooms >= 4:
            if 79.5 >= total_area >= 140:
                return '1564812'
            elif 68.5 >= total_area > 79.5:
                return '1564801'
            elif 46.5 >= total_area >= 51.5:
                return '1568734'        
            else:
                return 'другие'
        elif rooms == 0:
            return 'другие'
    else:
        return building_series_id_y
        

            
data['building_series_id_y'] = data.apply(lambda x: change_building_series_id(x['building_series_id_y'], x['total_area'], x['rooms']), axis = 1)
data.groupby('building_series_id_y').count()

In [ ]:
data['ceiling_height'] = round(data['ceiling_height'], 1)

In [ ]:
print(data['ceiling_height'].unique())

In [ ]:
head_ceiling_height = ['ceiling_height', 'locality_name', 'price_bym']
data_ceiling_height= pd.DataFrame(data = data, columns = (head_ceiling_height))
print(data_ceiling_height.head(5))

In [ ]:
data_ceiling_height.dropna(inplace=True)
data_ceiling_height = data_ceiling_height.loc[data_ceiling_height.loc[:, 'locality_name'] == 'Санкт-Петербург']
data_ceiling_height.sort_values(by='ceiling_height', inplace=True)
data_ceiling_height.head(10)

In [ ]:
data_ceiling_height['ceiling_height'] = pd.to_numeric(data_ceiling_height['ceiling_height'])

In [ ]:
data_ceiling_height = data_ceiling_height.query('ceiling_height not in [1, 14, 100, 1.8]')
def ceiling_height_chng (ceiling_height):
    if 100 == ceiling_height:
        return 0
    elif ceiling_height < 2:
        return 0
    elif 10 < ceiling_height <= 19:
        return 0
    elif 20 <= ceiling_height <= 32:
        return ceiling_height / 10
    else:
        return ceiling_height

    
data_ceiling_height['ceiling_height'] = data_ceiling_height.apply(lambda x: ceiling_height_chng(x['ceiling_height']), axis = 1)

In [ ]:
data_ceiling_height.plot(x='ceiling_height', y='price_bym', kind='scatter', alpha=0.09)

In [ ]:
data_ceiling_height = data_ceiling_height.query('ceiling_height < 5')
data_ceiling_height = data_ceiling_height.query('price_bym < 700000')
ceiling_height_mean = data_ceiling_height['ceiling_height'].mean()
data_ceiling_height.plot(x='ceiling_height', y='price_bym', kind='scatter', alpha=0.09)
print(data_ceiling_height['ceiling_height'].corr(data_ceiling_height['price_bym']))

In [ ]:
data['ceiling_height'] = data_ceiling_height.apply(lambda x: ceiling_height_chng(x['ceiling_height']), axis = 1)
data['ceiling_height'] = data['ceiling_height'].fillna(0)
data['ceiling_height'].astype('int')
data['ceiling_height'] = round(data['ceiling_height'], 1)
print(data['ceiling_height'].unique())

In [ ]:
data_tmp = data.query('ceiling_height != 0')

In [ ]:
ceiling_height_by_local_dict = data_tmp.groupby('building_series_id_y')['ceiling_height'].agg('mean').to_dict()
data['ceiling_height_mean'] = data['building_series_id_y'].map(ceiling_height_by_local_dict)
data.loc[data['ceiling_height'] == 0, 'ceiling_height'] = data.loc[data['ceiling_height'] == 0, 'ceiling_height_mean']
data[['ceiling_height', 'ceiling_height_mean', 'building_series_id_y']].head(10)


In [ ]:
data.drop(['ceiling_height_mean'], axis='columns', inplace=True)
data.plot(x='ceiling_height', y='price_bym', kind='scatter', alpha=0.09)
print(data['ceiling_height'].corr(data['price_bym']))

In [ ]:
data['living_area'] = data['living_area'].fillna(0)
data['living_area'].describe()
data['living_to_total_area'] = data['living_area'] / data['total_area']
data['living_to_total_area'].describe()

In [ ]:
living_to_total_area_by_bldngid = data.groupby('building_series_id_y')['living_to_total_area'].agg('mean').to_dict()
data['living_to_total_area_mean'] = data['building_series_id_y'].map(living_to_total_area_by_bldngid)
data['living_to_total_area'] = data['living_to_total_area'].where(data['living_to_total_area'] > 0.29, data['living_to_total_area_mean'])
data['living_to_total_area'] = data['living_to_total_area'].where(data['living_to_total_area'] < 0.81, data['living_to_total_area_mean'])
data[['living_to_total_area', 'living_to_total_area_mean', 'building_series_id_y']].head(10)

In [ ]:
data.drop(['living_to_total_area_mean'], axis='columns', inplace=True)
data['rooms'].unique()

In [ ]:
print(data['is_apartment'].unique())
print(data['is_apartment'].value_counts())

In [ ]:
data['kitchen_area'].fillna(0, inplace = True)
data['kitchen_area'].astype('int')
data['kitchen_area'] = round(data['kitchen_area'], 0)
print(data['kitchen_area'].unique())

In [ ]:
print(data['kitchen_area'].hist())

In [ ]:
print(data['kitchen_area'].hist(range=(0, 20)))

In [ ]:
head_kitchen_area = ['kitchen_area', 'locality_name', 'price_bym']
data_kitchen_area = pd.DataFrame(data = data, columns = (head_kitchen_area))
print(data_kitchen_area.head(5))

In [ ]:
data_kitchen_area = (data_kitchen_area
                    .query('kitchen_area >= 5 and kitchen_area <= 20 and locality_name == "Санкт-Петербург"')
)
data_kitchen_area.groupby(by='kitchen_area').count()

In [ ]:
data_kitchen_area.plot(x='kitchen_area', y='price_bym', kind='scatter', alpha=0.09)
print(data_kitchen_area['kitchen_area'].corr(data['price_bym']))

In [ ]:
data_kitchen_area = data_kitchen_area.query('price_bym <= 200000')
data_kitchen_area.plot(x='kitchen_area', y='price_bym', kind='scatter', alpha=0.09)
print(data['kitchen_area'].corr(data['price_bym']))

In [ ]:
data_tmp = data.query('kitchen_area > 5')

In [ ]:
kitchen_area_by_local_dict = data_tmp.groupby('building_series_id_y')['kitchen_area'].agg('mean').to_dict()
data['kitchen_area_mean'] = data['building_series_id_y'].map(ceiling_height_by_local_dict)
data.loc[data['kitchen_area'] == 0, 'kitchen_area'] = data.loc[data['kitchen_area'] == 0, 'kitchen_area_mean']
data[['kitchen_area', 'kitchen_area_mean', 'building_series_id_y']].head(10)

In [ ]:
data.drop(['kitchen_area_mean'], axis='columns', inplace=True)
data['kitchen_to_total'] = data['kitchen_area'] / data['total_area']

In [ ]:
print(data['balcony'].unique())

In [ ]:
#data.groupby('balcony').count()
data_tmp = data.query('balcony == "nan"')
data_tmp.groupby('building_series_id_y').agg({'price_bym':['count']})

In [ ]:
data['balcony'] = data['balcony'].fillna(-1)

In [ ]:
data.plot(x='balcony', y='price_bym', kind='scatter', alpha=0.09)
print(data['balcony'].corr(data['price_bym']))

In [ ]:
print(data['airports_nearest'].describe())

In [ ]:
data['floors_total'] = data['floors_total'].fillna(-1)
data['floors_total'].unique()

In [ ]:
def floor_type_ins (floor, floors_total):
    if floor == 1:
        return 'first'
    elif floor == floors_total:
        return 'last'
    else:
        return 'другой'

    
data['floor_type'] = data.apply(lambda x: floor_type_ins(x['floor'], x['floors_total']), axis = 1)
data.groupby('floor_type').count()

In [ ]:
data['time_publication'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%d')
data['month_expstn'] = pd.DatetimeIndex(data['first_day_exposition']).month
data['day_expstn'] = pd.DatetimeIndex(data['first_day_exposition']).weekday
data['year_expstn'] = pd.DatetimeIndex(data['first_day_exposition']).year
data['day_expstn'].unique()

In [ ]:
dataspb = data.query('locality_name == "Санкт-Петербург" and cityCenters_nearest > 0')
dataspb.plot(y='year_expstn', kind='hist', grid=True, legend=True)
dataspb2017 = dataspb.query('year_expstn == 2017')
dataspb2018 = dataspb.query('year_expstn == 2018')

In [ ]:
dataspb2018.describe()

In [ ]:
dataspb2017.describe()

In [ ]:
data['airports_nearest'].hist()

In [ ]:
data_airports_nearest = data.copy()
data_airports_nearest.dropna(subset = ['airports_nearest'], inplace=True)
data.info()

In [ ]:
data_airports_nearest.plot(x='airports_nearest', y='price_bym', kind='scatter', alpha=0.09)
print(data['airports_nearest'].corr(data['price_bym']))

In [ ]:
print(data['cityCenters_nearest'].describe())

In [ ]:
dataspb = data.query('locality_name == "Санкт-Петербург" and cityCenters_nearest > 0')
dataspb.groupby('cityCenters_nearest')['locality_name'].count()
dataspb.info()

In [ ]:
dataspb = data.query('locality_name == "Санкт-Петербург" and cityCenters_nearest > 0')
dataspb.groupby('cityCenters_nearest')['locality_name'].count()
print(dataspb['cityCenters_nearest'].hist(bins=28, range=(0, 28000)))
print(dataspb['cityCenters_nearest'].hist(range=(3000, 8000)))
print(dataspb['cityCenters_nearest'].hist(range=(11000, 16000)))

In [ ]:
dataspb.plot(x='cityCenters_nearest', y='price_bym', kind='scatter', alpha=0.09)
print(dataspb['cityCenters_nearest'].corr(data['price_bym']))

In [ ]:
dataspb_1564812 = dataspb.query('building_series_id_y == "1564812" and price_bym <= 500000')    
dataspb_1564812.plot(x='cityCenters_nearest', y='price_bym', kind='scatter', alpha=0.09)
print(dataspb_1564812['cityCenters_nearest'].corr(data['price_bym']))

In [ ]:
print(data['parks_around3000'].unique())
data['parks_around3000'].hist()

In [ ]:
dataspb.plot(x='parks_around3000', y='price_bym', kind='scatter', alpha=0.09)
print(dataspb['parks_around3000'].corr(data['price_bym']))

In [ ]:
print(data['parks_nearest'].describe())
data['parks_nearest'].hist()

In [ ]:
dataspb.plot(x='parks_nearest', y='price_bym', kind='scatter', alpha=0.09)
print(dataspb['parks_nearest'].corr(data['price_bym']))

In [ ]:
print(data['ponds_nearest'].describe())
data['ponds_nearest'].hist()

In [ ]:
dataspb.plot(x='ponds_nearest', y='price_bym', kind='scatter', alpha=0.09)
print(dataspb['ponds_nearest'].corr(data['price_bym']))

In [ ]:
data['last_price'] = data['last_price'].astype('int')
data['floors_total'] = data['floors_total'].astype('int')
data['floor'] = data['floor'].astype('int')
data['is_apartment'] = data['is_apartment'].astype('bool')
data['balcony'] = data['balcony'].astype('int')
data['price_bym'] = data['price_bym'].astype('int')

In [ ]:
data['last_price'] = data['last_price'].astype('int')
data['floors_total'] = data['floors_total'].astype('int')
data['floor'] = data['floor'].astype('int')
data['is_apartment'] = data['is_apartment'].astype('bool')
data['balcony'] = data['balcony'].astype('int')
data['price_bym'] = data['price_bym'].astype('int')

In [ ]:
data['total_area'].plot(kind='hist', bins=30, grid=True, legend=True)

In [ ]:
data['total_area'].plot(kind='hist', bins=30, grid=True, legend=True, range=(10, 200))

In [ ]:
data['price_bym'].plot(kind='hist', bins=30, grid=True, legend=True)

In [ ]:
data['price_bym'].hist(bins=30, range=(30, 280))

In [ ]:
data_groupby_building_series_id = data.groupby('building_series_id_y').count()
data_groupby_building_series_id.plot(kind='pie', y='total_area', legend=False)

In [ ]:
data['days_exposition'].describe()

In [ ]:
data_standart = data.query('total_area > 20 and total_area < 200 and price_bym > 30 and price_bym < 250 and ceiling_height < 5 and kitchen_area < 20')
data_standart.info()

In [ ]:
data_standart['price_bym'].hist()

In [ ]:
data_standart1 = data_standart.query('building_series_id_y == "1564812" and locality_name == "Санкт-Петербург"')

In [ ]:
data_standart1_first = data_standart1.loc[data_standart1.loc[:,'floor_type'] == 'first']
data_standart1_last = data_standart1.loc[data_standart1.loc[:,'floor_type'] == 'last']
data_standart1_other = data_standart1.loc[data_standart1.loc[:,'floor_type'] == 'другой']

In [ ]:
data_standart1_first['price_bym'].describe()

In [ ]:
data_standart1_last['price_bym'].describe()

In [ ]:
data_standart1_other['price_bym'].describe()

In [ ]:
head_table = ['rooms', 'ceiling_height', 'kitchen_area', 'cityCenters_nearest', 'living_to_total_area', 'kitchen_to_total', 'month_expstn', 'day_expstn', 'year_expstn']
i = 0
for i in range(len(head_table)):
    print(head_table[i], data_standart1[head_table[i]].corr(data['price_bym']))
    print(data_standart1.pivot_table(index = head_table[i], values='price_bym', aggfunc = 'describe'))
    i += 1


In [ ]:
i = 0
for i in range(len(head_table)):
    data_standart1.plot(x=head_table[i], y='price_bym', kind='scatter', alpha=0.09)
    i += 1

In [ ]:
print("Влияние на цену удалённости от центра города", data_standart1['price_bym'].corr(data['cityCenters_nearest']))
print("Влияние на цену высоты потолков", data_standart1['price_bym'].corr(data['ceiling_height']))
print("Влияние на цену площади кухни", data_standart1['price_bym'].corr(data['kitchen_area']))

In [ ]:
data_locality = data.pivot_table(index = 'locality_name', values='price_bym', aggfunc = ['median', 'count'])
data_locality.columns = ['median', 'count']
data_locality.sort_values(by='count', ascending=False, inplace=True)
data_locality.drop(['Санкт-Петербург'], inplace=True)
data_locality.head(10)

In [ ]:
#dataspb = dataspb.query('building_series_id_y == "1564812"')
print(dataspb['cityCenters_nearest'].hist(bins=28, range=(0, 28000)))
print(dataspb['cityCenters_nearest'].hist(range=(3000, 8000)))
print(dataspb['cityCenters_nearest'].hist(range=(11000, 16000)))

In [ ]:
dataspb_citycenter3_8 = dataspb.query('cityCenters_nearest > 3000 and cityCenters_nearest < 8000')
dataspb_citycenter11_16 = dataspb.query('cityCenters_nearest > 11000 and cityCenters_nearest < 16000')
dataspb_citycenter3_8['total_area'].hist(bins=25, range=(0, 250))
dataspb_citycenter3_8['total_area'].hist(bins=25, range=(40, 100))

In [ ]:
dataspb_citycenter11_16['total_area'].hist(bins=15, range=(0, 150))
dataspb_citycenter11_16['total_area'].hist(bins=15, range=(30, 70))

In [ ]:
dataspb_citycenter3_8_rooms = dataspb_citycenter3_8.pivot_table(index='rooms', values='price_bym', aggfunc = ['median', 'count'])
dataspb_citycenter11_16_rooms = dataspb_citycenter11_16.pivot_table(index='rooms', values='price_bym', aggfunc = ['median', 'count'])
dataspb_citycenter3_8_rooms

In [ ]:
dataspb_citycenter11_16_rooms

In [ ]:
dataspb_citycenter3_8_total_40_100 = dataspb_citycenter3_8.query('total_area > 40 and total_area < 100')
dataspb_citycenter11_16_rms1_ttl_30_70 = dataspb_citycenter11_16.query('rooms == 1 and total_area > 30 and total_area < 70')
dataspb_citycenter11_16_rms2_ttl_30_70 = dataspb_citycenter11_16.query('rooms == 2 and total_area > 30 and total_area < 70')
dataspb_citycenter11_16_rms3_ttl_30_70 = dataspb_citycenter11_16.query('rooms == 3 and total_area > 30 and total_area < 70')

In [ ]:
def del_columns (df_for_del):
    pricebym = df_for_del.drop(['total_area', 'rooms', 'total_images', 'last_price', 'ceiling_height', 'floors_total', 'living_area', 'floor', 'studio', 'area', 'parks_nearest', 'ponds_around3000',  'ponds_nearest', 'days_exposition', 'living_to_total_area', 'kitchen_to_total', 'month_expstn', 'day_expstn', 'year_expstn', 'open_plan', 'kitchen_area', 'balcony', 'airports_nearest', 'cityCenters_nearest', 'parks_around3000'], axis='columns')
    pricebym = pricebym.drop(['ceiling_height', 'parks_nearest', 'ponds_around3000',  'ponds_nearest', 'days_exposition', 'living_to_total_area', 'kitchen_to_total', 'month_expstn', 'day_expstn', 'year_expstn', 'open_plan', 'kitchen_area','cityCenters_nearest', 'parks_around3000'])
    return pricebym

data_price_by_m = dataspb_citycenter3_8_total_40_100.corr()
data_price_by_m_only = del_columns(data_price_by_m) 
data_price_by_m_only

In [ ]:
data_price_by_m1 = dataspb_citycenter11_16_rms1_ttl_30_70.corr()
data_price_by_m1_only = del_columns(data_price_by_m1) 
data_price_by_m1_only

In [ ]:
data_price_by_m2 = dataspb_citycenter11_16_rms2_ttl_30_70.corr()
data_price_by_m2_only = del_columns(data_price_by_m2) 
data_price_by_m2_only

In [ ]:
data_price_by_m3 = dataspb_citycenter11_16_rms3_ttl_30_70.corr()
data_price_by_m3_only = del_columns(data_price_by_m3) 
data_price_by_m3_only



In [ ]:
data_appatment = data.query('is_apartment == True')
#data_appatment['cityCenters_nearest'].hist()
data_appatment['cityCenters_nearest'].hist(range=(10500, 16500))

In [ ]:
data_appatment['ceiling_height'].hist(range=(2.5, 2.8))

In [ ]:
data_appatment['kitchen_area'].hist(range=(7, 11))

In [ ]:
data_appatment = data_appatment.query('cityCenters_nearest >= 10500 and cityCenters_nearest <= 16500 and ceiling_height >= 2.5 and ceiling_height <= 2.8 and kitchen_area >= 7 and kitchen_area <= 11')
print("Влияние на цену удалённости от центра города", data_appatment['price_bym'].corr(data_appatment['cityCenters_nearest']))
print("Влияние на цену высоты потолков", data_appatment['price_bym'].corr(data_appatment['ceiling_height']))
print("Влияние на цену площади кухни", data_appatment['price_bym'].corr(data_appatment['kitchen_area']))

In [ ]:
data_studio = data.query('studio == True')
#data_appatment['cityCenters_nearest'].hist()
data_studio['cityCenters_nearest'].hist(range=(12000, 18000))

In [ ]:
data_studio['ceiling_height'].hist(range=(2.6, 2.8))

In [ ]:
data_studio = data_studio.query('cityCenters_nearest >= 12000 and cityCenters_nearest <= 18000 and ceiling_height >= 2.6 and ceiling_height <= 2.8')
print("Влияние на цену удалённости от центра города", data_studio['price_bym'].corr(data_studio['cityCenters_nearest']))
print("Влияние на цену высоты потолков", data_studio['price_bym'].corr(data_studio['ceiling_height']))

In [ ]:
data_openplan = data.query('open_plan == True')
#data_openplan['cityCenters_nearest'].hist()
data_openplan['cityCenters_nearest'].hist(range=(11500, 14000))

In [ ]:
data_openplan['ceiling_height'].hist(range=(2.7, 2.8))

In [ ]:
data_openplan['kitchen_area'].hist(range=(2.8, 2.8))

In [ ]:
data_openplan = data_openplan.query('cityCenters_nearest >= 11500 and cityCenters_nearest <= 14000 and ceiling_height >= 2.7 and ceiling_height <= 2.8 and kitchen_area >= 2.7 and kitchen_area <= 2.9')
print("Влияние на цену удалённости от центра города", data_openplan['price_bym'].corr(data_openplan['cityCenters_nearest']))
